In [58]:
import imp
import sys
sys.path.append('../lib')
from outlier import *
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from itertools import groupby
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import os

In [2]:
store_cible=pd.HDFStore('../PCA/cible_crossB2B_1.h5')
uselessFeatures=list(pd.read_table('./uselessFeatures.csv', sep=',')['name'])
dg=store_cible.select('df201606_sample200000')
cible=dg.fin
dg=dg.drop(uselessFeatures,axis=1)
dg_drop=dg.drop('fin',axis=1)

In [3]:
dg_drop_minmax=pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(dg_drop),columns=dg_drop.columns)

In [49]:
normal_dataset=np.array(dg_drop_minmax)[np.where(np.array(dg['fin'])==1)]

In [50]:
pipeline=Pipeline([ ('removeOutliers',OutliersKmeans(parallel=True,kmeans=kmeans,showbar=True))])

In [51]:
fittedpipeline=pipeline.fit(normal_dataset)

In [55]:
fittedpipeline.named_steps['removeOutliers'].iextreme_values

array([False, False, False, ...,  True,  True,  True], dtype=bool)

# Kmeans

In [47]:
result_file='result_grid_kmeans.csv'
dir_path='./'

### Sans réduction de dimension

In [56]:
text_file = open(os.path.join(dir_path, result_file), 'w')
normal_dataset=np.array(dg_drop_minmax)[np.where(np.array(dg['fin'])==1)]
for k in range(3,12):
    
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10,random_state=5)
    pipeline=Pipeline([ ('removeOutliers',OutliersKmeans(parallel=True,kmeans=kmeans,showbar=True))])


    tuned_parameters = [{'penalty': ['l1','l2'],
            'C':list(np.logspace(-4, 4, 10))}]


    clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=10)
    fittedpipeline=pipeline.fit(normal_dataset)
    data=fittedpipeline.transform(np.array(dg_drop_minmax))
    cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]
    clf.fit(data,cible_dropt)


    print('--------------------------', file=text_file)
    print("K :"+str(k), file=text_file)

    print('', file=text_file)
    print("Best parameters set found on development set:", file=text_file)
    print('', file=text_file)
    print(clf.best_params_, file=text_file)
    print('', file=text_file)

    print("Grid scores on development set:", file=text_file)
    print('', file=text_file)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params), file=text_file)
    print('', file=text_file)

text_file.close()

100%|#########################################################################|


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   1.1s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   1.1s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s


...

[CV] ............. penalty=l2, C=0.0001, score=0.391011, total=   1.6s
[CV] penalty=l2, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   13.1s remaining:    0.0s


[CV] ............. penalty=l2, C=0.0001, score=0.367137, total=   1.6s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   1.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   1.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   1.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   1.1s
[CV] penalty=l1, C=0.000774263682681 .................................
[CV] .. penalty=l1, C=0.000774263682681, score=0.500000, total=   1.3s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .. penalty=l2, C=0.000774263682681, score=0.406215, total=   2.2s
[CV] penalty=l2, C=0.000774263682681 .................................
[CV] .

[CV] ...... penalty=l2, C=21.5443469003, score=0.817550, total=  19.9s
[CV] penalty=l2, C=21.5443469003 .....................................
[CV] ...... penalty=l2, C=21.5443469003, score=0.815546, total=  16.3s
[CV] penalty=l2, C=21.5443469003 .....................................
[CV] ...... penalty=l2, C=21.5443469003, score=0.775300, total=  18.0s
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.790433, total= 3.5min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.829734, total= 3.4min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.814758, total= 2.6min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] ....... penalty=l1, C=166.81005372, score=0.815763, total= 2.9min
[CV] penalty=l1, C=166.81005372 ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 91.5min finished


### Avec réduction de dimension 

In [87]:
result_file='result_grid_kmeans_reduction.csv'
dir_path='./'

In [ ]:
text_file = open(os.path.join(dir_path, result_file), 'w')
normal_dataset=np.array(dg_drop_minmax)[np.where(np.array(dg['fin'])==1)]
for k in range(3,15):
    for ncompo in range(2,10):
    
        kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10,random_state=5)
        pipeline=Pipeline([('dim_red',PCA(n_components=ncompo,svd_solver='arpack')),
                           ('removeOutliers',OutliersKmeans(parallel=True,kmeans=kmeans,showbar=False))])


        tuned_parameters = [{'penalty': ['l1','l2'],
                'C':list(np.logspace(-4, 4, 10))}]


        clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=1)
        fittedpipeline=pipeline.fit(normal_dataset)
        data=fittedpipeline.transform(np.array(dg_drop_minmax)) 
        cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]
        clf.fit(data,cible_dropt)


        print('--------------------------', file=text_file)
        print("K :"+str(k), file=text_file)
        print("Number component :"+str(ncompo), file=text_file)

        print('', file=text_file)
        print("Best parameters set found on development set:", file=text_file)
        print('', file=text_file)
        print(clf.best_params_, file=text_file)
        print('', file=text_file)

        print("Grid scores on development set:", file=text_file)
        print('', file=text_file)
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params), file=text_file)
        print('', file=text_file)

text_file.close()

Fitting 5 folds for each of 20 candidates, totalling 100 fits


# Mahanalobis

In [77]:
result_file='result_grid_mh.csv'
dir_path='./'

In [79]:
text_file = open(os.path.join(dir_path, result_file), 'w')

for k in range(2,15):
    
    for supportf in [0.95,0.99,0.995]:

        tuned_parameters = [{'penalty': ['l1','l2'],
                'C':list(np.logspace(-4, 5, 15))}]

        pipeline=Pipeline([('dim_red',PCA(n_components=k,svd_solver='arpack'))
                   ,('removeOutliers',OutlierMahalanobis(verbose=False,qqplot=False,support_fraction=supportf))])


        clf = GridSearchCV(LogisticRegression(max_iter=10000), tuned_parameters, scoring='roc_auc', cv=5,verbose=1)
        fittedpipeline=pipeline.fit(np.array(dg_drop_minmax))

        data=fittedpipeline.transform(np.array(dg_drop_minmax))
        cible_dropt=cible[~fittedpipeline.named_steps['removeOutliers'].iextreme_values]

        clf.fit(data,cible_dropt)


        print('--------------------------', file=text_file)
        print("Components :"+str(k), file=text_file)
        print("Support Fraction :"+str(supportf), file=text_file)

        print('', file=text_file)
        print("Best parameters set found on development set:", file=text_file)
        print('', file=text_file)
        print(clf.best_params_, file=text_file)
        print('', file=text_file)

        print("Grid scores on development set:", file=text_file)
        print('', file=text_file)
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                % (mean, std * 2, params), file=text_file)
        print('', file=text_file)

text_file.close()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.0s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.1s
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.1s
[CV] penalty=l1, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.6s remaining:    0.0s


[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.1s
[CV] penalty=l1, C=0.0001 ............................................
[CV] ............. penalty=l1, C=0.0001, score=0.500000, total=   0.1s
[CV] penalty=l2, C=0.0001 ............................................


...

[CV] ............. penalty=l2, C=0.0001, score=0.665592, total=   0.1s
[CV] penalty=l2, C=0.0001 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.9s remaining:    0.0s


[CV] ............. penalty=l2, C=0.0001, score=0.671511, total=   0.1s
[CV] penalty=l1, C=0.000439397056076 .................................
[CV] .. penalty=l1, C=0.000439397056076, score=0.500000, total=   0.1s
[CV] penalty=l1, C=0.000439397056076 .................................
[CV] .. penalty=l1, C=0.000439397056076, score=0.500000, total=   0.2s
[CV] penalty=l1, C=0.000439397056076 .................................
[CV] .. penalty=l1, C=0.000439397056076, score=0.500000, total=   0.1s
[CV] penalty=l1, C=0.000439397056076 .................................
[CV] .. penalty=l1, C=0.000439397056076, score=0.500000, total=   0.2s
[CV] penalty=l1, C=0.000439397056076 .................................
[CV] .. penalty=l1, C=0.000439397056076, score=0.500000, total=   0.1s
[CV] penalty=l2, C=0.000439397056076 .................................
[CV] .. penalty=l2, C=0.000439397056076, score=0.662292, total=   0.3s
[CV] penalty=l2, C=0.000439397056076 .................................
[CV] .

[CV] ..... penalty=l2, C=0.719685673001, score=0.774216, total=   0.3s
[CV] penalty=l2, C=0.719685673001 ....................................
[CV] ..... penalty=l2, C=0.719685673001, score=0.762262, total=   0.3s
[CV] penalty=l2, C=0.719685673001 ....................................
[CV] ..... penalty=l2, C=0.719685673001, score=0.708182, total=   0.2s
[CV] penalty=l1, C=3.16227766017 .....................................
[CV] ...... penalty=l1, C=3.16227766017, score=0.751964, total=   0.2s
[CV] penalty=l1, C=3.16227766017 .....................................
[CV] ...... penalty=l1, C=3.16227766017, score=0.778492, total=   0.2s
[CV] penalty=l1, C=3.16227766017 .....................................
[CV] ...... penalty=l1, C=3.16227766017, score=0.774390, total=   0.2s
[CV] penalty=l1, C=3.16227766017 .....................................


KeyboardInterrupt: 